In [1]:
# Paso 1: Conectar con Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Paso 2: Instalar y actualizar las librerías (Método Minimalista y Correcto)
print("\nInstalando y actualizando librerías...")
!pip install --upgrade -q mne pytorch-lightning timm
print("✅ Librerías listas.")


Instalando y actualizando librerías...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.3 MB/s eta 0:00:00
 

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
import pytorch_lightning as pl
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score, MulticlassCohenKappa
import numpy as np

# ==============================================================================
# DEFINICIÓN DE LA ARQUITECTURA DEL MODELO
# ==============================================================================
def get_convnext_model(num_classes=5, pretrained=True):
    model = timm.create_model('convnextv2_tiny.fcmae_ft_in22k_in1k', pretrained=pretrained)
    original_conv = model.stem[0]
    new_first_conv = nn.Conv2d(1, original_conv.out_channels,
                               kernel_size=original_conv.kernel_size,
                               stride=original_conv.stride,
                               padding=original_conv.padding,
                               bias=(original_conv.bias is not None))
    with torch.no_grad():
        new_first_conv.weight[:, :] = original_conv.weight.clone().mean(dim=1, keepdim=True)
    model.stem[0] = new_first_conv
    num_ftrs = model.head.fc.in_features
    model.head.fc = nn.Linear(num_ftrs, num_classes)
    return model

# ==============================================================================
# DEFINICIÓN DEL CLASIFICADOR (LÓGICA DE ENTRENAMIENTO)
# ==============================================================================
class SleepStageClassifier(pl.LightningModule):
    def __init__(self, model_name='convnextv2_tiny', num_classes=5, learning_rate=1e-4, class_weights=None):
        super().__init__()
        self.save_hyperparameters()
        self.model = get_convnext_model(num_classes=num_classes, pretrained=True)

        # Métricas para cada fase (versión corregida y compatible)
        self.train_metrics = nn.ModuleDict({
            'acc': MulticlassAccuracy(num_classes=num_classes),
            'f1': MulticlassF1Score(num_classes=num_classes)
        })
        self.val_metrics = nn.ModuleDict({
            'acc': MulticlassAccuracy(num_classes=num_classes),
            'f1': MulticlassF1Score(num_classes=num_classes)
        })
        self.test_metrics = nn.ModuleDict({
            'acc': MulticlassAccuracy(num_classes=num_classes),
            'f1': MulticlassF1Score(num_classes=num_classes),
            'kappa': MulticlassCohenKappa(num_classes=num_classes)
        })

        self.weights = torch.tensor(class_weights, dtype=torch.float) if class_weights is not None else None
        self.loss_fn = F.cross_entropy

    def forward(self, x):
        return self.model(x)

    def _common_step(self, batch, batch_idx):
        x, y_true = batch
        y_pred_logits = self(x)
        if self.weights is not None:
            loss = self.loss_fn(y_pred_logits, y_true, weight=self.weights.to(self.device))
        else:
            loss = self.loss_fn(y_pred_logits, y_true)
        preds = torch.argmax(y_pred_logits, dim=1)
        return loss, preds, y_true

    def training_step(self, batch, batch_idx):
        loss, preds, y_true = self._common_step(batch, batch_idx)
        metrics = {name: metric(preds, y_true) for name, metric in self.train_metrics.items()}
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log_dict({f'train_{k}': v for k, v in metrics.items()}, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, preds, y_true = self._common_step(batch, batch_idx)
        metrics = {name: metric(preds, y_true) for name, metric in self.val_metrics.items()}
        self.log('val_loss', loss, prog_bar=True)
        self.log_dict({f'val_{k}': v for k, v in metrics.items()}, on_epoch=True)

    def test_step(self, batch, batch_idx):
        loss, preds, y_true = self._common_step(batch, batch_idx)
        metrics = {name: metric(preds, y_true) for name, metric in self.test_metrics.items()}
        self.log('test_loss', loss)
        self.log_dict({f'test_{k}': v for k, v in metrics.items()})

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.hparams.learning_rate)

print("✅ Clases y funciones del modelo definidas exitosamente.")

✅ Clases y funciones del modelo definidas exitosamente.


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from pathlib import Path
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import logging
import numpy as np

# ==============================================================================
# ¡NUEVA LÍNEA DE OPTIMIZACIÓN PARA LA GPU!
# ==============================================================================
# Esto activa el modo de alto rendimiento para las GPUs modernas como la L4/A100.
torch.set_float32_matmul_precision('medium')
# ==============================================================================

# --- Dataset optimizado para la memoria y con filtrado de etiquetas ---
class SleepEDFXDataset(Dataset):
    def __init__(self, parquet_dir):
        self.file_paths = sorted(list(Path(parquet_dir).glob('*.parquet')))
        data_frames = [pd.read_parquet(f) for f in self.file_paths]
        self.full_df = pd.concat(data_frames, ignore_index=True)

        # Filtrado para evitar el error "CUDA device-side assert"
        valid_labels = [0, 1, 2, 3, 4]
        original_rows = len(self.full_df)
        self.full_df = self.full_df[self.full_df['label'].isin(valid_labels)]
        filtered_rows = len(self.full_df)

        logging.info(f"Dataset cargado con {original_rows} épocas en total.")
        logging.info(f"Se eliminaron {original_rows - filtered_rows} épocas con etiquetas inválidas.")
        logging.info(f"Número final de épocas para el entrenamiento: {filtered_rows}")

    def __len__(self):
        return len(self.full_df)

    def __getitem__(self, idx):
        row = self.full_df.iloc[idx]
        spectrogram_flat = row[:-1].values.astype(np.float32)
        label = np.int64(row['label'])
        spectrogram_2d = spectrogram_flat.reshape(1, 76, 60)
        return torch.from_numpy(spectrogram_2d), torch.tensor(label)

# --- Rutas y Configuración ---
sleep_edfx_processed_dir = Path('/content/drive/MyDrive/sleep_edfx_processed/')
checkpoint_dir = Path('/content/drive/MyDrive/sleep_model_checkpoints/')
checkpoint_dir.mkdir(parents=True, exist_ok=True)

# --- Preparar los datos ---
full_dataset = SleepEDFXDataset(sleep_edfx_processed_dir)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=2, persistent_workers=True)

# --- Configurar el Entrenamiento ---
class_weights = [0.7, 3.5, 0.5, 1.5, 1.2]
model = SleepStageClassifier(learning_rate=1e-4, class_weights=class_weights)
checkpoint_callback = ModelCheckpoint(
    dirpath=str(checkpoint_dir),
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min'
)
trainer = pl.Trainer(
    accelerator='gpu',
    devices=1,
    max_epochs=20,
    callbacks=[checkpoint_callback],
    logger=pl.loggers.TensorBoardLogger('/content/drive/MyDrive/sleep_logs/', name='convnext_v2_colab_pro')
)

# --- ¡INICIAR EL ENTRENAMIENTO! ---
logging.info("===================================================")
logging.info("--- Iniciando el entrenamiento del modelo... ---")
logging.info("===================================================")
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
logging.info("========== ENTRENAMIENTO COMPLETADO ==========")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type       | Params | Mode 
-----------------------------------------------------
0 | model         | ConvNeXt   | 27.9 M | train
1 | train_metrics | ModuleDict | 0      | train
2 | val_metrics   | ModuleDict | 0      | train
3 | test_metrics  | ModuleDict | 0      | train
-----------------------------------------------------
27.9 M    Trainable params
0         Non-trainable params
27.9 M    Total params
111.469   Total estimated model params size (MB)
259       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 17, global step 51480: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 18, global step 54340: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 19, global step 57200: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
